# Loading data

In [2]:
import pandas as pd
order_products__train = pd.read_csv("../instacart_2017_05_01/order_products__train.csv")
order_products__prior = pd.read_csv("../instacart_2017_05_01/order_products__prior.csv")
products = pd.read_csv("../instacart_2017_05_01/products.csv")
orders = pd.read_csv("../instacart_2017_05_01/orders.csv")

In [4]:
orders['eval_set'].unique()

array(['prior', 'train', 'test'], dtype=object)

# Train data

In [5]:
orders_train = orders[orders['eval_set'] == 'train'].reset_index()

In [6]:
orders_train.head()

,index,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,10,1187899,1,train,11,4,8,14.0
1,25,1492625,2,train,15,1,11,30.0
2,49,2196797,5,train,5,0,11,6.0
3,74,525192,7,train,21,2,11,6.0
4,78,880375,8,train,4,1,14,10.0


In [7]:
orders_train = orders_train[['user_id', 'order_id']]

In [13]:
orders_train['order_id'].unique().shape[0]

131209

In [14]:
order_products__train['order_id'].unique().shape[0]

131209

In [15]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [16]:
order_products__train = order_products__train[['order_id', 'product_id', 'reordered']]

In [17]:
order_products__train = order_products__train.groupby('order_id')['product_id'].apply(list).reset_index()

In [18]:
order_products__train.head()

,order_id,product_id
0,1,"[49302, 11109, 10246, 49683, 43633, 13176, 472..."
1,36,"[39612, 19660, 49235, 43086, 46620, 34497, 486..."
2,38,"[11913, 18159, 4461, 21616, 23622, 32433, 2884..."
3,96,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]"
4,98,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."


In [19]:
user_products__train = pd.merge(order_products__train, orders_train, on='order_id')

In [21]:
user_products__train.head()

,order_id,product_id,user_id
0,1,"[49302, 11109, 10246, 49683, 43633, 13176, 472...",112108
1,36,"[39612, 19660, 49235, 43086, 46620, 34497, 486...",79431
2,38,"[11913, 18159, 4461, 21616, 23622, 32433, 2884...",42756
3,96,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]",17227
4,98,"[8859, 19731, 43654, 13176, 4357, 37664, 34065...",56463


In [22]:
user_products__train = user_products__train[['user_id', 'product_id']]